In [1]:
from collections import defaultdict

class TwoSAT:
    def __init__(self, n):
        self.n = n  # Número de variables
        self.graph = defaultdict(list)  # Grafo de implicaciones
        self.graph_inv = defaultdict(list)  # Grafo invertido para SCC

    def add_clause(self, a, b):
        """
        Agrega una cláusula (a v b) al sistema.
        Args:
            a (int): Literal 1 (positivo o negativo).
            b (int): Literal 2 (positivo o negativo).
        """
        # a v b equivale a (!a -> b) y (!b -> a)
        self.graph[-a].append(b)
        self.graph[-b].append(a)
        self.graph_inv[b].append(-a)
        self.graph_inv[a].append(-b)

    def kosaraju(self):
        """
        Encuentra las SCC usando el algoritmo de Kosaraju.
        Returns:
            list[list[int]]: Lista de SCC (componentes fuertemente conexas).
        """
        visited = set()
        stack = []
        order = []

        # Primera pasada: orden topológico
        def dfs1(node):
            visited.add(node)
            for neighbor in self.graph[node]:
                if neighbor not in visited:
                    dfs1(neighbor)
            order.append(node)

        for i in range(-self.n, self.n + 1):
            if i != 0 and i not in visited:
                dfs1(i)

        # Segunda pasada: encontrar SCC
        visited.clear()
        scc = []

        def dfs2(node, component):
            visited.add(node)
            component.append(node)
            for neighbor in self.graph_inv[node]:
                if neighbor not in visited:
                    dfs2(neighbor, component)

        for node in reversed(order):
            if node not in visited:
                component = []
                dfs2(node, component)
                scc.append(component)

        return scc

    def solve(self):
        """
        Resuelve el problema 2-SAT.
        Returns:
            dict or None: Asignación satisfactoria o None si no es satisfacible.
        """
        scc = self.kosaraju()
        assignment = {}

        # Verificar si hay conflictos en las SCC
        position = {}
        for i, component in enumerate(scc):
            for node in component:
                position[node] = i

        for i in range(1, self.n + 1):
            if position[i] == position[-i]:
                return None  # No es satisfacible

        # Asignar valores
        for component in scc:
            for node in component:
                if node not in assignment:
                    assignment[node] = True
                    assignment[-node] = False

        return {i: assignment[i] for i in range(1, self.n + 1)}


# Ejemplo: Resolver 2-SAT
solver = TwoSAT(3)  # Tres variables: x1, x2, x3

# Agregar cláusulas (x1 v ¬x2), (¬x1 v x3), (x2 v ¬x3)
solver.add_clause(1, -2)
solver.add_clause(-1, 3)
solver.add_clause(2, -3)

resultado = solver.solve()
if resultado:
    print("Fórmula satisfacible. Asignación:")
    for variable, valor in resultado.items():
        print(f"x{variable} = {valor}")
else:
    print("Fórmula no satisfacible.")


Fórmula satisfacible. Asignación:
x1 = True
x2 = True
x3 = True
